# Basic classification: Classify images of clothing
A tensorflow serving style service example using BentoML


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=tensorflow&ea=tensorflow_2_fashion_mnist&dt=tensorflow_2_fashion_mnist)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# add venv PATH to shell command PATH
import sys, os
if sys.base_prefix not in os.environ['PATH']:
    os.environ['PATH'] = f"{sys.base_prefix}/bin:{os.environ['PATH']}"

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import io

# TensorFlow
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(_train_images, train_labels), (_test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
train_images = _train_images / 255.0
test_images = _test_images / 255.0

In [ ]:
class FashionMnist(tf.keras.Model):
    def __init__(self):
        super(FashionMnist, self).__init__()
        self.cnn = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    
    @staticmethod
    def image_bytes2tensor(inputs):
        inputs = tf.map_fn(lambda i: tf.io.decode_png(i, channels=1), inputs, dtype=tf.uint8)
        inputs = tf.cast(inputs, tf.float32)
        inputs = (255.0 - inputs) / 255.0
        inputs = tf.reshape(inputs, [-1, 28, 28])
        return inputs

    @tf.function(input_signature=[tf.TensorSpec(shape=(None,), dtype=tf.string)])
    def predict_image(self, inputs):
        inputs = self.image_bytes2tensor(inputs)
        return self(inputs)
    
    def call(self, inputs):
        return self.cnn(inputs)


## test the image preprocessing

In [ ]:
# pick up a test image
d_test_img = _test_images[0]
print(class_names[test_labels[0]])

plt.imshow(255.0 - d_test_img, cmap='gray')
plt.imsave("test.png", 255.0 - d_test_img, cmap='gray')

# read bytes
with open("test.png", "rb") as f:
    img_bytes = f.read()


## train the model


In [ ]:
model = FashionMnist()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1)

## test the model

In [ ]:
predict = model.predict_image(tf.constant([img_bytes] * 3))
klass = tf.argmax(predict, axis=1)
[class_names[k] for k in klass]

And the model predicts a label as expected.

# Define & save BentoService

In [ ]:
%%writefile tensorflow_fashion_mnist.py

import base64
import bentoml
import tensorflow as tf
import numpy as np
from PIL import Image

from bentoml.artifact import (
    TensorflowSavedModelArtifact,
)
from bentoml.handlers import TensorflowTensorHandler, ClipperStringsHandler


FASHION_MNIST_CLASSES = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


@bentoml.env(pip_dependencies=['tensorflow', 'numpy', 'pillow'])
@bentoml.artifacts([TensorflowSavedModelArtifact('model')])
class FashionMnistTensorflow(bentoml.BentoService):

    @bentoml.api(TensorflowTensorHandler)
    def predict(self, inputs):
        outputs = self.artifacts.model.predict_image(inputs)
        output_classes = tf.math.argmax(outputs, axis=1)
        return [FASHION_MNIST_CLASSES[o] for o in output_classes]

    @bentoml.api(ClipperStringsHandler)
    def predict_clipper(self, strings):
        _bytes = [base64.b64decode(i) for i in strings]
        inputs = tf.constant(_bytes, dtype=tf.string)
        outputs = self.artifacts.model.predict_image(inputs)
        output_classes = tf.math.argmax(outputs, axis=1)
        return [FASHION_MNIST_CLASSES[o] for o in output_classes]

In [ ]:
from tensorflow_fashion_mnist import FashionMnistTensorflow

bento_svc = FashionMnistTensorflow()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

# Build & Run Bento Service in Docker

In [28]:
# replace PIP_INDEX_URL with your prefer pypi mirror
NAME = saved_path.split('/')[-1].lower()
!docker build -t {NAME} \
    --build-arg PIP_TRUSTED_HOST=192.168.138.2 \
    --build-arg PIP_INDEX_URL=http://192.168.138.2/simple \
    {saved_path}

Sending build context to Docker daemon  1.818MB
Step 1/15 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/15 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> 040e53e25c33
Step 3/15 : EXPOSE 5000
 ---> Using cache
 ---> ca6dd0ed4ba4
Step 4/15 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 307260816d47
Step 5/15 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> 8c85ff8665b4
Step 6/15 : COPY . /bento
 ---> c3a21764a080
Step 7/15 : WORKDIR /bento
 ---> Running in f273dbddc7e3
Removing intermediate container f273dbddc7e3
 ---> d8a26ae62991
Step 8/15 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 9ca13251c5e7
Removing intermediate container 9ca13251c5e7
 ---> a29c2e4bb099
Step 9/15 : RUN conda env update -n base -f /bento/environment.yml
 ---

In [29]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)

!docker run -itd -p {PORT}:5000 --cpus 1 -e FLAGS="--workers 1 --enable-microbatch" {NAME}:latest

33289
f65cc547fb968a1864b6ac0180644cb77cd6734e6ae5112b6fd4d5fd967c3d8f


# Test with requests

In [31]:
import base64
import json
import requests

with open("test.png", "rb") as f:
    img_bytes = f.read()
img_b64 = base64.b64encode(img_bytes).decode()


headers = {"content-type": "application/json"}
data = json.dumps(
       {"instances": [{"b64": img_b64}] * 3}
)

json_response = requests.post(f'http://127.0.0.1:{PORT}/predict', data=data, headers=headers)
print(json_response)
print(json_response.text)

<Response [200]>
["Ankle boot", "Ankle boot", "Ankle boot"]


# Benchmark with locust

In [47]:
%%writefile benchmark_bentoml_mb.py
from locust import HttpLocust, TaskSet, task, constant
from functools import lru_cache

import math
import random
import numpy as np
import pandas as pd
import json
import base64
import requests


@lru_cache(maxsize=1)
def data_producer():

    with open("test.png", "rb") as f:
        img_bytes = f.read()
    img_b64 = base64.b64encode(img_bytes).decode()

    def _gen_data(size=3):
        headers = {"content-type": "application/json"}
        data = json.dumps(
               {"instances": [{"b64": img_b64}] * size}
        )
        return headers, data

    return _gen_data


class WebsiteTasks(TaskSet):

    @staticmethod
    def get_data():
        headers, data = data_producer()(1)
        return headers, data
        
    @task
    def index(self):
        headers, data = self.get_data()
        self.client.post("/predict", data, headers=headers)

class WebsiteUser(HttpLocust):
    task_set = WebsiteTasks
    wait_time = constant(1)

Writing benchmark_bentoml_mb.py


In [34]:
!locust -f benchmark_bentoml_mb.py -H http://127.0.0.1:{PORT}

[2020-02-20 21:39:26,897] beta/INFO/locust.main: Starting web monitor at http://*:8089
[2020-02-20 21:39:26,898] beta/INFO/locust.main: Starting Locust 0.14.4
[2020-02-20 21:39:45,927] beta/INFO/locust.runners: Hatching and swarming 1000 users at the rate 20 users/s (0 users already running)...
^C
[2020-02-20 21:46:49,696] beta/ERROR/stderr: KeyboardInterrupt
[2020-02-20 21:46:49,696] beta/ERROR/stderr: 2020-02-20T13:46:49Z
[2020-02-20 21:46:49,696] beta/ERROR/stderr: 
[2020-02-20 21:46:49,696] beta/INFO/locust.main: Shutting down (exit code 0), bye.
[2020-02-20 21:46:49,696] beta/INFO/locust.main: Cleaning up runner...
[2020-02-20 21:46:49,696] beta/INFO/locust.main: Running teardowns...
 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s failures/s
--------------------------------------------------------------------------------------------------------------------------------------------
 POST /predict          